In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [3]:
df = pd.read_csv('boston.csv')
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [5]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [8]:
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.svm import SVR 
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [9]:
gbr = GradientBoostingRegressor() 
svr = SVR() 
rf = RandomForestRegressor()
lr = LinearRegression()

In [10]:
from sklearn.model_selection import cross_val_score
X = x
print("Gradient Boost",np.mean(cross_val_score(gbr,X,y,cv=10))) 
print("Linear Regression",np.mean(cross_val_score(lr,X,y,cv=10))) 
print("SVM",np.mean(cross_val_score(svr,X,y,cv=10)))
print("Random Forest",np.mean(cross_val_score(rf,X,y,cv=10)))

Gradient Boost 0.4520098975471708
Linear Regression 0.20252899006056796
SVM -0.4109699157425082
Random Forest 0.49541401087746795


In [11]:
from sklearn.model_selection import GridSearchCV
params = { 
          'loss':['ls', 'lad', 'huber', 'quantile'], 
          'learning_rate':[0.1,0.5,1.0], 
          'n_estimators':[10,50,100,200,300], 
          'subsample':[0.5,0.75,1.0] }
grid = GridSearchCV(estimator=GradientBoostingRegressor(),param_grid=params,cv=10)
grid.fit(x,y)
bestScore = grid.best_score_
bestParam = grid.best_params_

c:\Users\abhinav.aggarwal\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
900 fits failed out of a total of 1800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
450 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\abhinav.aggarwal\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\abhinav.aggarwal\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1344, in wrapper
    estimator._validate_params()
  File "c:\Users\abhinav.aggarwal\AppData\Local\Programs\Py

In [12]:
print(bestScore)
print(bestParam)

0.5888183860183634
{'learning_rate': 0.1, 'loss': 'huber', 'n_estimators': 300, 'subsample': 0.5}


In [13]:
gdBoost2 = GradientBoostingRegressor(**bestParam)
gdBoost2.fit(x_train,y_train)
y_pred = gdBoost2.predict(x_test)

In [14]:
from sklearn.metrics import r2_score
print("Final r2 score for gradient boosting is: ",r2_score(y_test,y_pred))
## Gradient descent also works on the basis of decision stump estimator

Final r2 score for gradient boosting is:  0.8936253069201432


In [28]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
param = {
    'estimator' : [DecisionTreeRegressor(),gdBoost2],
    'n_estimators': [10, 50, 100],
    'max_samples': [0.5, 0.8, 1.0],
    'max_features': [0.5, 0.8, 1.0],
}
bagging = BaggingRegressor()
grid_search = GridSearchCV(estimator=bagging, param_grid=param, cv=5)
grid_search.fit(x_test, y_test)
bestBagParam = grid_search.best_params_
bestBagScore = grid_search.best_score_


In [29]:
print(bestBagParam)
print(bestBagScore)

{'estimator': DecisionTreeRegressor(), 'max_features': 1.0, 'max_samples': 0.8, 'n_estimators': 10}
0.7264906471082424


In [32]:
newModel = BaggingRegressor(**bestBagParam)
newModel.fit(x_train,y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(), max_samples=0.8)

In [33]:
yPredNew = newModel.predict(x_test)
print("r2 score is ",r2_score(y_test,yPredNew))

r2 score is  0.8023970276055243
